In [5]:
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO
from urllib.request import urlopen
from tqdm import tqdm
import time
import csv
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [6]:
def getTables(response):
    # HTTPステータスコードが200（成功）の場合のみ処理を続行
    if response.getcode() == 200:
        # HTMLをパースしてBeautifulSoupオブジェクトを作成
        bs = BeautifulSoup(response, 'html.parser')
        bs = bs.decode('UTF-8')
        html_string = str(bs)
        html_io = StringIO(html_string)

        # テーブルデータを抽出
        tables = pd.read_html(html_io)
        df = tables[0]
        print(df)
        return df

    else:
        print(f"HTTPステータスコード {response.getcode()}: ページの取得に失敗しました")
        

In [7]:
#開催年
years = [str(i).zfill(4) for i in range(2021, 2022)]
#開催場所 01:札幌, 02:函館, 03:福島, 04:新潟, 05:東京, 06:中山, 07:中京, 08:京都, 09::阪神, 10:小倉
places = [str(i).zfill(2) for i in range (5, 6)]
#開催回
times = [str(i).zfill(2) for i in range(1, 2)]
#開催日
days = [str(i).zfill(2) for i in range(1, 2)]
#レースNo
races = [str(i).zfill(2) for i in range(1, 13)]

raceIdList = []
for y in years:
    for p in places:
        for t in times:
            for d in days:
                for r in races:
                    raceIdList.append(y + p + t + d + r)
                    
print(raceIdList)


['202105010101', '202105010102', '202105010103', '202105010104', '202105010105', '202105010106', '202105010107', '202105010108', '202105010109', '202105010110', '202105010111', '202105010112']


In [8]:
url = 'https://db.netkeiba.com/race/'
df = pd.DataFrame()
for raceId in tqdm(raceIdList):
    response = urlopen(url+raceId)
    race_data = getTables(response)
    race_data['raceId'] = [raceId] * len(race_data)
    df = pd.concat([df,race_data],axis=0)
     
    time.sleep(3)

print(df)

  0%|          | 0/12 [00:00<?, ?it/s]

    着  順  枠  番  馬  番         馬名  性齢  斤量    騎手     タイム     着差     単勝  人  気  \
0      1     1     2    プレフェリータ  牝3  54  戸崎圭太  1:25.5    NaN   10.2     5   
1      2     2     4    ナンノコレシキ  牝3  54  横山武史  1:25.6    3/4    3.9     2   
2      3     2     3   ホープケツァール  牝3  54  丸田恭介  1:26.1      3    8.1     3   
3      4     4     8     フクノルッカ  牝3  54  田辺裕信  1:26.3      1    2.1     1   
4      5     6    12    シンシアハート  牝3  54  大野拓弥  1:26.5  1.1/4   13.0     6   
5      6     8    16   ファインヒューズ  牝3  54  内田博幸  1:26.6    1/2   16.1     7   
6      7     5    10     アウリッシュ  牝3  54  三浦皇成  1:26.7      1    8.6     4   
7      8     7    13       ドナリノ  牝3  51   原優介  1:26.9    3/4  147.7    12   
8      9     8    15    タミルナードゥ  牝3  54  杉原誠人  1:27.4      3  174.7    13   
9     10     1     1   キタノコンキスタ  牝3  54  嶋田純次  1:27.4     ハナ  219.6    15   
10    11     3     6    ボンバーガール  牝3  54  武士沢友  1:28.3      5  104.8     8   
11    12     6    11       ナイアレ  牝3  54   吉田豊  1:28.5  1.1/4  119.2    10   

  8%|▊         | 1/12 [00:03<00:36,  3.30s/it]

    着  順  枠  番  馬  番         馬名  性齢  斤量    騎手     タイム     着差     単勝  人  気  \
0      1     4     8    トゥーパンクス  牡3  56  内田博幸  1:25.5    NaN   10.3     6   
1      2     6    11  タマモアンドロメダ  牡3  56   武藤雅  1:26.0      3    6.8     2   
2      3     2     3     ランボボーイ  牡3  54  ルメール  1:26.1    1/2    2.5     1   
3      4     3     6   ファインハッピー  牝3  54  三浦皇成  1:26.4  1.3/4    9.3     5   
4      5     6    12      アナマリー  牝3  54  北村宏司  1:26.5    3/4    8.6     4   
5      6     4     7    ヴィントラオフ  牡3  56  石川裕紀  1:26.9  2.1/2  137.1    14   
6      7     3     5   ミライヘノイブキ  牡3  56  伊藤工真  1:27.2  1.1/2   22.3     8   
7      8     1     2        ツカサ  セ3  56  野中悠太  1:27.5  1.3/4   74.4    12   
8      9     5     9  アクアマリンブルー  牡3  56  嶋田純次  1:27.5     クビ   91.4    13   
9     10     7    13   シルバーファーン  牡3  56  田辺裕信  1:27.6    1/2   12.1     7   
10    11     1     1  サクラトップクロス  牡3  53  小林脩斗  1:27.7    1/2   28.3     9   
11    12     8    15  クリスタルローレル  牡3  56  蓑島靖典  1:28.0      2  164.8    15   

 17%|█▋        | 2/12 [00:06<00:34,  3.44s/it]

...

    着  順  枠  番  馬  番         馬名  性齢  斤量    騎手     タイム     着差    単勝  人  気  \
0      1     5     6       ポタジェ  牡4  55  川田将雅  1:59.0    NaN   2.0     1   
1      2     8    13   サンレイポケット  牡6  56  横山武史  1:59.1     クビ   3.5     2   
2      3     5     7       フランツ  牡6  56  Ｍ．デム  1:59.1     クビ  26.6     6   
3      4     4     4  アフリカンゴールド  セ6  56  田辺裕信  1:59.2    1/2  63.3    10   
4      5     3     3       ギベオン  牡6  59  戸崎圭太  1:59.4  1.1/4  39.8     7   
5      6     6     8    ビターエンダー  牡4  55  津村明秀  1:59.4     ハナ   7.7     3   
6      7     7    10    オウケンムーン  牡6  57   吉田豊  1:59.4     クビ  85.1    13   
7      8     7    11    レッドサイオン  セ5  56  北村宏司  1:59.5     クビ  73.9    11   
8      9     1     1     ゴールドギア  牡6  56  三浦皇成  1:59.6    1/2  61.4     9   
9     10     4     5  アトミックフォース  牡5  57   武藤雅  1:59.7    1/2  12.0     5   
10    11     6     9  マイネルファンロン  牡6  57  柴田大知  1:59.9  1.1/2  82.9    12   
11    12     2     2   リンディーホップ  牝6  54  大野拓弥  2:00.1  1.1/4  55.5     8   
12    13    

 92%|█████████▏| 11/12 [00:38<00:03,  3.55s/it]

    着  順  枠  番  馬  番         馬名  性齢  斤量    騎手     タイム     着差     単勝  人  気  \
0      1     6    11      ヤシャマル  牡4  56  横山武史  1:48.5    NaN    2.2     1   
1      2     7    13    ダイワクンナナ  牝4  54  田辺裕信  1:48.7  1.1/4   11.6     5   
2      3     4     7     フィリアーノ  牡5  57  津村明秀  1:49.0  1.3/4   29.3     8   
3      4     7    12   ウインアルカンナ  牝4  54  石川裕紀  1:49.0    アタマ   34.2     9   
4      5     8    14  ビバヴィットーリオ  牡4  56  川田将雅  1:49.1    1/2   10.2     4   
5      6     8    15   レットミーアウト  牝4  54  柴田大知  1:49.2     クビ   13.0     6   
6      7     2     2    コスモスタック  牡4  56   武藤雅  1:49.2     ハナ  121.4    12   
7      8     1     1   サクラヴァルール  牡4  56  戸崎圭太  1:49.4  1.1/2   17.1     7   
8      9     3     5     レベンディス  セ4  56  三浦皇成  1:49.4     クビ    6.3     3   
9     10     4     6    レッドサーシャ  牝4  54  内田博幸  1:49.5    1/2   77.1    11   
10    11     2     3     クィーンビー  牝8  55  野中悠太  1:49.6    3/4  207.5    13   
11    12     5     8  レインカルナティオ  牡4  56  ルメール  1:49.7    1/2    4.4     2   

100%|██████████| 12/12 [00:42<00:00,  3.52s/it]

    着  順  枠  番  馬  番         馬名  性齢  斤量    騎手     タイム     着差     単勝  人  気  \
0      1     1     2    プレフェリータ  牝3  54  戸崎圭太  1:25.5    NaN   10.2     5   
1      2     2     4    ナンノコレシキ  牝3  54  横山武史  1:25.6    3/4    3.9     2   
2      3     2     3   ホープケツァール  牝3  54  丸田恭介  1:26.1      3    8.1     3   
3      4     4     8     フクノルッカ  牝3  54  田辺裕信  1:26.3      1    2.1     1   
4      5     6    12    シンシアハート  牝3  54  大野拓弥  1:26.5  1.1/4   13.0     6   
..   ...   ...   ...        ...  ..  ..   ...     ...    ...    ...   ...   
10    11     2     3     クィーンビー  牝8  55  野中悠太  1:49.6    3/4  207.5    13   
11    12     5     8  レインカルナティオ  牡4  56  ルメール  1:49.7    1/2    4.4     2   
12    13     6    10   ローズパルファン  牝4  51   原優介  1:50.0  1.1/2   50.8    10   
13    14     3     4   アトムアストレア  牝6  55  木幡巧也  1:50.2  1.1/4  402.3    15   
14    15     5     9  オールイズスパーブ  牝5  55  嶋田純次  1:51.6      9  337.5    14   

         馬体重        調教師        raceId  
0    468(-4)   [東]  萩原清  2021050101

In [9]:
df.to_csv('data.csv',encoding='utf-8')
df.columns
print(df)

    着  順  枠  番  馬  番         馬名  性齢  斤量    騎手     タイム     着差     単勝  人  気  \
0      1     1     2    プレフェリータ  牝3  54  戸崎圭太  1:25.5    NaN   10.2     5   
1      2     2     4    ナンノコレシキ  牝3  54  横山武史  1:25.6    3/4    3.9     2   
2      3     2     3   ホープケツァール  牝3  54  丸田恭介  1:26.1      3    8.1     3   
3      4     4     8     フクノルッカ  牝3  54  田辺裕信  1:26.3      1    2.1     1   
4      5     6    12    シンシアハート  牝3  54  大野拓弥  1:26.5  1.1/4   13.0     6   
..   ...   ...   ...        ...  ..  ..   ...     ...    ...    ...   ...   
10    11     2     3     クィーンビー  牝8  55  野中悠太  1:49.6    3/4  207.5    13   
11    12     5     8  レインカルナティオ  牡4  56  ルメール  1:49.7    1/2    4.4     2   
12    13     6    10   ローズパルファン  牝4  51   原優介  1:50.0  1.1/2   50.8    10   
13    14     3     4   アトムアストレア  牝6  55  木幡巧也  1:50.2  1.1/4  402.3    15   
14    15     5     9  オールイズスパーブ  牝5  55  嶋田純次  1:51.6      9  337.5    14   

         馬体重        調教師        raceId  
0    468(-4)   [東]  萩原清  2021050101

In [19]:
dataset = df[['着  順','枠  番', '馬  番', '斤量', '単勝', '人  気','raceId']]
dataset['raceId'] = dataset['raceId'].astype('category')
dataset['ans'] = dataset['着  順'].map(lambda x:1 if(x<=5) else 0)

trainList = dataset['raceId'].unique()[:10]
testList = dataset['raceId'].unique()[10:]
train = dataset[dataset['raceId'].isin(trainList)]
test = dataset[dataset['raceId'].isin(testList)]
train_x = train[[ '枠  番', '馬  番', '斤量', '単勝', '人  気','raceId']]
train_y = train['ans']
test_x = test[[ '枠  番', '馬  番', '斤量', '単勝', '人  気','raceId']]
test_y = test['ans']


C:\Users\yamay\AppData\Local\Temp\ipykernel_6488\3577131448.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['raceId'] = dataset['raceId'].astype('category')
C:\Users\yamay\AppData\Local\Temp\ipykernel_6488\3577131448.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['ans'] = dataset['着  順'].map(lambda x:1 if(x<=5) else 0)


In [20]:
model = lgb.LGBMClassifier()
model.fit(train_x, train_y)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 50, number of negative: 100
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000222 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 112
[LightGBM] [Info] Number of data points in the train set: 150, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.333333 -> initscore=-0.693147
[LightGBM] [Info] Start training from score -0.693147
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

LGBMClassifier()

In [21]:
pred = model.predict(test_x)
test['pred'] = pred
test

C:\Users\yamay\AppData\Local\Temp\ipykernel_6488\3523114529.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred'] = pred


,着 順,枠 番,馬 番,斤量,単勝,人 気,raceId,ans,pred
0,1,5,6,55,2.0,1,202105010111,1,1
1,2,8,13,56,3.5,2,202105010111,1,1
2,3,5,7,56,26.6,6,202105010111,1,0
3,4,4,4,56,63.3,10,202105010111,1,0
4,5,3,3,59,39.8,7,202105010111,1,0
5,6,6,8,55,7.7,3,202105010111,0,1
6,7,7,10,57,85.1,13,202105010111,0,0
7,8,7,11,56,73.9,11,202105010111,0,0
8,9,1,1,56,61.4,9,202105010111,0,0
9,10,4,5,57,12.0,5,202105010111,0,0
